# Baseline Policy

In [1]:
using Revise
using Random
using Printf
using DataStructures
using Flux
using POMDPs
using POMDPModelTools
using POMDPSimulators
using BeliefUpdaters
using POMDPPolicies
using DiscreteValueIteration
using MDPModelChecking
using StaticArrays
using RLInterface
using DeepQLearning
using AutomotiveDrivingModels
using AutomotivePOMDPs
using AutomotiveSensors
using LocalApproximationValueIteration
using Reel
using AutoViz
using ProgressMeter
using JLD2
using FileIO
using BSON
using PedCar

loaded


In [2]:
includet("../src/masking.jl")
includet("../src/masked_dqn.jl")
includet("../src/qmdp_approximation.jl")
includet("../src/decomposed_tracking.jl")
includet("../src/decomposition.jl")
includet("../src/baseline_policy.jl")
includet("../src/util.jl")
includet("../src/render_helpers.jl")

In [3]:
rng = MersenneTwister(1);
cam = FitToContentCamera(0.);

## Scenario

In [4]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =[VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

In [5]:
mdp = PedCarMDP(env=env, pos_res=2.0, vel_res=2., ped_birth=0.7, car_birth=0.7);
init_transition!(mdp);

In [6]:
pomdp = UrbanPOMDP(env=env,
                   sensor = PerfectSensor(),
                   ego_goal = LaneTag(2, 1),
                       obs_dist = ObstacleDistribution(mdp.env, 
                                                   upper_obs_pres_prob=0., 
                                                   left_obs_pres_prob=0.0, 
                                                   right_obs_pres_prob=0.0),
                   max_cars=1, 
                   max_peds=1, 
                   car_birth=0.7, 
                   ped_birth=0.7, 
                   max_obstacles=1., # no fixed obstacles
                   lidar=false,
                   ego_start=20,
                   ΔT=0.1);
# instantiate sub problems
## CAR POMDP FOR TRACKING 1 CAR
car_pomdp = deepcopy(pomdp)
car_pomdp.models = pomdp.models
car_pomdp.max_peds = 0
car_pomdp.max_cars = 1
## PED POMDP FOR TRACKING 1 PEDESTRIAN
ped_pomdp = deepcopy(pomdp)
ped_pomdp.models = pomdp.models
ped_pomdp.max_peds = 1
ped_pomdp.max_cars = 0
## PEDCAR POMDP FOR THE POLICY (Model checking + DQN)
pedcar_pomdp = deepcopy(pomdp)
pedcar_pomdp.models = pomdp.models # shallow copy!
pedcar_pomdp.max_peds = 1
pedcar_pomdp.max_cars = 1
pedcar_pomdp.max_obstacles = 0

0

## Initialize Baseline Policy

Input: a driving scene, use `obs_to_scene` to convert the observation vector

In [7]:
@load "../pc_util_processed_low.jld2" qmat util pol
safe_policy = ValueIterationPolicy(mdp, qmat, util, pol);
threshold = 0.99
mask = SafetyMask(mdp, safe_policy, threshold);

In [8]:
ego_model = get_ego_baseline_model(env);

In [9]:
masked_baseline = MaskedEgoBaseline(pomdp, pedcar_pomdp, ego_model, mask, UrbanAction[]);
baseline = EgoBaseline(pomdp, ego_model);

## Evaluation

**Observation as input**

In [11]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, baseline, PreviousObservationUpdater(), n_ep=1000, max_steps=400, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

Progress: 100%|█████████████████████████████████████████|  ETA: 0:00:00

182.739922 seconds (291

Progress: 100%|█████████████████████████████████████████| Time: 0:03:02


.57 M allocations: 22.820 GiB, 5.00% gc time)
Summary for 1000 episodes: 
Average reward: 0.006 
Average # of steps: 171.019 
Average # of violations: 0.000 


**RNN Belief Updater**

In [10]:
n_models = 5
car_models = Vector{Chain}(undef, n_models)
ped_models = Vector{Chain}(undef, n_models)
for i=1:n_models
    car_models[i] = BSON.load("../RNNFiltering/model_car_$i.bson")[:model] 
    Flux.loadparams!(car_models[i], BSON.load("../RNNFiltering/weights_car_$i.bson")[:weights])
    ped_models[i] = BSON.load("../RNNFiltering/model_ped_$i.bson")[:model]
    Flux.loadparams!(ped_models[i], BSON.load("../RNNFiltering/weights_ped_$i.bson")[:weights])
end
pres_threshold = 0.3
ref_updaters = Dict(AgentClass.PEDESTRIAN => SingleAgentTracker(ped_pomdp, ped_models, pres_threshold, VehicleDef()),
                AgentClass.CAR =>  SingleAgentTracker(car_pomdp, car_models, pres_threshold, VehicleDef()))
up = MultipleAgentsTracker(pomdp, ref_updaters, Dict{Int64, SingleAgentTracker}());

In [16]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, masked_baseline, up, n_ep=1000, max_steps=400, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

Progress: 100%|█████████████████████████████████████████|  ETA: 0:00:04

3747.766191 seconds (8.04 G allocations: 1.741 TiB, 17.93% gc time)


Progress: 100%|█████████████████████████████████████████| Time: 1:02:27


Summary for 1000 episodes: 
Average reward: 0.005 +/- 0.008 
Average # of steps: 94.971 +/- 8.566 
Average # of violations: 0.100 +/- 3.162 


## Visualize Baseline Policy

In [11]:
function POMDPModelTools.generate_sori(pomdp::UrbanPOMDP, s::Scene, a::UrbanAction, rng::AbstractRNG)
    sp, o, r = generate_sor(pomdp, s, a, rng)
    return sp, o, r, deepcopy(pomdp.models)
end

In [13]:
hr = HistoryRecorder(rng=rng, max_steps=400)
s0 = initialstate(pomdp, rng)
# updater = PreviousObservationUpdater()
o0 = generate_o(pomdp, s0, UrbanAction(0.), s0, rng)
b0 = initialize_belief(up, o0)
@time hist = simulate(hr, pomdp, masked_baseline, up, b0, s0);

  6.218647 seconds (10.91 M allocations: 1.919 GiB, 13.93% gc time)


In [17]:
function find_collision(pomdp, test_policy, rng)
    up = PreviousObservationUpdater()
    @showprogress for ep=1:10000
        hr = HistoryRecorder(rng=rng, max_steps=400)
        s0 = initialstate(pomdp, rng)
        o0 = generate_o(pomdp, s0, UrbanAction(0.), s0, rng)
        b0 = initialize_belief(up, o0)
        hist2 = simulate(hr, pomdp, test_policy, up, b0, s0)
        if sum(hist2.reward_hist .< 0.) != 0.
            println("Crash")
            return hist2
        end
    end
    return hist2 
end
hist = find_collision(pomdp, baseline, rng);

Progress:   0%|                                         |  ETA: 0:36:55

InterruptException: InterruptException:

In [14]:
animate_history(hist, pomdp,
                action_overlays = a -> [TextOverlay(text = ["Acc: $(a.acc) m/s^2"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3., 8.), incameraframe=true)],                                   
#                 ainfo_overlays = ai -> [GaussianSensorOverlay(sensor=GaussianSensor(), o=[veh for veh in ai[1] if veh.id != EGO_ID])],
#                     ainfo_overlays = ai -> [TextOverlay(text = ["Acc: $(ai[2]) m/s^2"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3., 6.), incameraframe=true)],
#                                         TextOverlay(text = ["Available Actions: $([a.acc for a in ai[2]])"], font_size=20,pos=VecE2(pomdp.env.params.x_min + 3.,10.), incameraframe=true)],
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [IDOverlay()],
                cam =  StaticCamera(VecE2(0., -8.), 12.0),
                speed_factor=2)

Frames{MIME{Symbol("image/png")}}("/tmp/tmpB7l2Dj", 0x000000000000005f, 20.0, nothing)

In [69]:
step = 57
s = hist.state_hist[step+1]
sb = hist.ainfo_hist[step][1]
model = hist.ainfo_hist[step][3]
models = hist.info_hist[step]
vehid = 1
vehm = model
cwm = model.crosswalk_drivers[3]
intm = model.intersection_driver

TTCIntersectionDriver
  a: LonAccelDirection
  navigator: RouteFollowingIDM
  intersection: Array{Lane}((2,))
  intersection_pos: VecSE2{Float64}
  ttc_threshold: Float64 3.0
  horizon: Float64 20.0
  stop_delta: Float64 4.0
  accel_tol: Float64 0.0
  priorities: Dict{Tuple{LaneTag,LaneTag},Bool}
  priority: Bool false
  stop: Bool false


In [73]:
m = deepcopy(vehm)
observe!(m, sb, pomdp.env.roadway, vehid)
m.a

LonAccelDirection(-9.0, 2)

In [74]:
for veh in s 
    println(veh)
end

Vehicle(101, VehicleState(VecSE2({6.041, 2.733}, 1.571), Frenet(RoadIndex({1, 0.695218}, {17, 1}), 9.733, -0.041, 0.000), 1.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(2, VehicleState(VecSE2({-1.500, -9.821}, -1.571), Frenet(RoadIndex({1, 0.223881}, {5, 1}), 5.821, 0.000, 0.000), 6.002), VehicleDef(CAR, 4.000, 1.800))
Vehicle(1, VehicleState(VecSE2({1.500, -10.000}, 1.571), Frenet(RoadIndex({1, 0.769231}, {6, 1}), 20.000, 0.000, 0.000), 0.000), VehicleDef(CAR, 4.000, 1.800))


In [75]:
for veh in sb
    println(veh)
end

Vehicle(1, VehicleState(VecSE2({1.500, -10.000}, 1.571), Frenet(RoadIndex({1, 0.769231}, {6, 1}), 20.000, 0.000, 0.000), 0.000), VehicleDef(CAR, 4.000, 1.800))
Vehicle(1, VehicleState(VecSE2({1.500, -10.000}, 1.571), Frenet(RoadIndex({1, 0.769231}, {6, 1}), 20.000, 0.000, 0.000), 0.000), VehicleDef(CAR, 4.000, 1.800))
Vehicle(2, VehicleState(VecSE2({-1.213, -9.108}, -1.594), Frenet(RoadIndex({1, 0.196468}, {5, 1}), 5.108, 0.287, -0.023), 6.929), VehicleDef(CAR, 4.000, 1.800))
Vehicle(101, VehicleState(VecSE2({6.156, 2.057}, 1.661), Frenet(RoadIndex({1, 0.646909}, {17, 1}), 9.057, -0.156, 0.090), 0.972), VehicleDef(PEDESTRIAN, 1.000, 1.000))


In [107]:
vehm.conflict_lanes

5-element Array{Lane,1}:
 Lane(LaneTag(5, 1), CurvePt[CurvePt({-1.500, -4.000, -1.571}, 0.000, NaN, NaN), CurvePt({-1.500, -30.000, -1.571}, 26.000, NaN, NaN)], 3.0, SpeedLimit(-Inf, Inf), LaneBoundary(:solid, :white), LaneBoundary(:solid, :white), LaneConnection[], LaneConnection[LaneConnection(U, CurveIndex(1, 0.000), RoadIndex({24, 1.000000}, {12, 1}), LaneConnection(U, CurveIndex(1, 0.000), RoadIndex({24, 1.000000}, {10, 1})])
 Lane(LaneTag(6, 1), CurvePt[CurvePt({1.500, -30.000, 1.571}, 0.000, NaN, NaN), CurvePt({1.500, -4.000, 1.571}, 26.000, NaN, NaN)], 3.0, SpeedLimit(-Inf, Inf), LaneBoundary(:solid, :white), LaneBoundary(:solid, :white), LaneConnection[LaneConnection(D, CurveIndex(1, 1.000), RoadIndex({1, 0.000000}, {15, 1}), LaneConnection(D, CurveIndex(1, 1.000), RoadIndex({1, 0.000000}, {13, 1})], LaneConnection[])      
 Lane(LaneTag(17, 1), CurvePt[CurvePt({6.000, -7.000, 1.571}, 0.000, 0.000, NaN), CurvePt({6.000, 7.000, 1.571}, 14.000, 0.000, NaN)], 4.0, SpeedLimit(-Inf

In [98]:
ped = s[findfirst(101, s)]
cos(ped.state.posF.ϕ)
ped.state.posF

Frenet(RoadIndex({1, 0.718562}, {19, 1}), 10.060, -0.530, 3.142)

In [41]:
cwm.yield

false

In [43]:
get_conflict_lanes(env.crosswalks[3], env.roadway)

5-element Array{Lane,1}:
 Lane(LaneTag(5, 1), CurvePt[CurvePt({-1.500, -4.000, -1.571}, 0.000, NaN, NaN), CurvePt({-1.500, -30.000, -1.571}, 26.000, NaN, NaN)], 3.0, SpeedLimit(-Inf, Inf), LaneBoundary(:solid, :white), LaneBoundary(:solid, :white), LaneConnection[], LaneConnection[LaneConnection(U, CurveIndex(1, 0.000), RoadIndex({24, 1.000000}, {12, 1}), LaneConnection(U, CurveIndex(1, 0.000), RoadIndex({24, 1.000000}, {10, 1})])
 Lane(LaneTag(6, 1), CurvePt[CurvePt({1.500, -30.000, 1.571}, 0.000, NaN, NaN), CurvePt({1.500, -4.000, 1.571}, 26.000, NaN, NaN)], 3.0, SpeedLimit(-Inf, Inf), LaneBoundary(:solid, :white), LaneBoundary(:solid, :white), LaneConnection[LaneConnection(D, CurveIndex(1, 1.000), RoadIndex({1, 0.000000}, {15, 1}), LaneConnection(D, CurveIndex(1, 1.000), RoadIndex({1, 0.000000}, {13, 1})], LaneConnection[])      
 Lane(LaneTag(17, 1), CurvePt[CurvePt({6.000, -7.000, 1.571}, 0.000, 0.000, NaN), CurvePt({6.000, 7.000, 1.571}, 14.000, 0.000, NaN)], 4.0, SpeedLimit(-Inf